## RNN Song Energy Changer
#### Author: Rhema Ike

In [23]:

## Predicting Energy Boosts
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn as sk

import functools
import random

In [3]:
#Load Data
# Loading Data

file_name = 'C:\\Users\\roike\\Documents\\research\\MachineDJ\\DataSets\\energy_boost_xy.xlsx'
mix_df = pd.read_excel(file_name)

num_of_keys = 12
convert_dict = {
                "tempo1": np.float32,
                "tempo2": np.float32,
                "y": np.int16
               }

for i in range(num_of_keys):
    dict_key = "key1"+str(i+1)
    convert_dict[dict_key] = np.int32

for i in range(num_of_keys):
    dict_key = "key2"+str(i+1)
    convert_dict[dict_key] = np.int32

mix_df = mix_df.astype(convert_dict)
mix_df.head()

,mode1,tempo1,key11,key12,key13,key14,key15,key16,key17,key18,...,key24,key25,key26,key27,key28,key29,key210,key211,key212,y
0,1,0.34188,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.34188,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.34188,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0.34188,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0.34188,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
df1 = mix_df[mix_df.columns[:14]]
df2 = mix_df[mix_df.columns[14:-1]]
ydf = mix_df[mix_df.columns[-1]]

In [14]:
ydf.head()

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int16

In [25]:
# Preprocessing Data
def pre_proc(df):
#     df1 = mix_df[mix_df.columns[:14]]
#     df2 = mix_df[mix_df.columns[14:-1]]
#     ydf = mix_df[mix_df.columns[-1]]
    
    seq_data = []
    data = []
    
    for row in df.values:
        seq_data.append([ [ row[:14], row[14:-1] ], int(row[-1]) ])
        
    random.shuffle(seq_data)
    
    boost = []
    no_boost = []
    
    for seq, target in seq_data:
        if target == 0:
            no_boost.append([seq,target])
        if target == 1:
            boost.append([seq,target])
            
    lower = min(len(no_boost), len(boost))
    
    boost = boost[:lower]
    no_boost = no_boost[:lower]
    
    seq_data = boost + no_boost
    random.shuffle(seq_data)
    
    X = []
    y = []
    
    for seq,target in seq_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X), np.array(y)


In [26]:
X,y = pre_proc(mix_df)

In [35]:
X.shape

(312, 2, 14)

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten

In [82]:
model = Sequential()

model.add(LSTM(68, input_shape=(X.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(68, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(2, activation='softmax'))

In [83]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'], 
)

In [85]:
model.fit(X,y, epochs=10,validation_split=0.1)

Train on 280 samples, validate on 32 samples
Epoch 1/10
280/280 [==============================] - 0s 132us/sample - loss: 0.6062 - accuracy: 0.6857 - val_loss: 0.7271 - val_accuracy: 0.5938
Epoch 2/10
280/280 [==============================] - 0s 114us/sample - loss: 0.5920 - accuracy: 0.6679 - val_loss: 0.7261 - val_accuracy: 0.5625
Epoch 3/10
280/280 [==============================] - 0s 114us/sample - loss: 0.5670 - accuracy: 0.7286 - val_loss: 0.7237 - val_accuracy: 0.5625
Epoch 4/10
280/280 [==============================] - 0s 110us/sample - loss: 0.5416 - accuracy: 0.7500 - val_loss: 0.6992 - val_accuracy: 0.6562
Epoch 5/10
280/280 [==============================] - 0s 110us/sample - loss: 0.5243 - accuracy: 0.7821 - val_loss: 0.7029 - val_accuracy: 0.6875
Epoch 6/10
280/280 [==============================] - 0s 114us/sample - loss: 0.4976 - accuracy: 0.8179 - val_loss: 0.6850 - val_accuracy: 0.6875
Epoch 7/10
280/280 [==============================] - 0s 125us/sample - loss: 0